# Import dependencies

In [41]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from time import strftime, gmtime
import subprocess
import time
import json

# Init S3 and Sagemaker Session

In [9]:
boto_session = boto3.session.Session() # To get general info about session
sagemaker_session= sagemaker.Session()

In [10]:
region = boto_session.region_name
region

'ap-southeast-1'

In [11]:
role = get_execution_role()
role

'arn:aws:iam::405392309789:role/sagemaker-risk-role'

# Define bucket for uploading model artifacts (Sagmaker Default Bucket)

In [12]:
default_bucket = sagemaker_session.default_bucket()

In [13]:
default_bucket

'sagemaker-ap-southeast-1-405392309789'

# Create s3 and sagemaker client instance

In [14]:
s3_client = boto_session.resource('s3') # Useful for accessing the s3 bucket
sm_client = boto3.client("sagemaker") # To get image uri, create endpoint and so on

# Define required component for migration

In [15]:
weight_data = 'model.joblib'
inference_script = 'inference.py'
zip_file = f"tar -cvpzf model.tar.gz {weight_data} {inference_script}"

# Execute shell subprocess to zip model artifacts

In [16]:
shell_process = subprocess.Popen(zip_file.split(), stdout=subprocess.PIPE)

In [17]:
prefix = f'ToMigration/{strftime("%Y-%m-%d-%H-%M-%S", gmtime())}'
model_artifacts = f"s3://{default_bucket}/{prefix}/model.tar.gz"

In [18]:
model_artifacts

's3://sagemaker-ap-southeast-1-405392309789/ToMigration/2023-06-06-20-32-31/model.tar.gz'

# Upload packed model artifacts to S3

In [19]:
response = s3_client.meta.client.upload_file('model.tar.gz', default_bucket, f'{prefix}/model.tar.gz')

# Get image URI from EC2 registry

In [20]:
image_uri = sagemaker.image_uris.retrieve(
                framework       = 'sklearn',
                region          = region,
                version         = '0.23-1',
                py_version      = 'py3',
                instance_type   = 'ml.t2.medium',
                image_scope     = 'inference'
            )

# Create Sagemaker Model (To be a medium for our pretrained weight)

In [21]:
model_name = "sm-sklearn" + "-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
create_model_resp = sm_client.create_model(
    ModelName   = model_name,
    Containers  = [
        {
            "Image"         : image_uri,
            "Mode"          : "SingleModel",
            "ModelDataUrl"  : model_artifacts,
            "Environment"   : {
                'SAGEMAKER_SUBMIT_DIRECTORY': model_artifacts,
                'SAGEMAKER_PROGRAM': 'inference.py'
            } 
        }
    ],
    ExecutionRoleArn = role,
)

In [22]:
create_model_resp

{'ModelArn': 'arn:aws:sagemaker:ap-southeast-1:405392309789:model/sm-sklearn-2023-06-06-20-32-36',
 'ResponseMetadata': {'RequestId': 'e7fb7d43-53d7-468f-b1b7-c1a60aa80ed8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e7fb7d43-53d7-468f-b1b7-c1a60aa80ed8',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '97',
   'date': 'Tue, 06 Jun 2023 20:32:36 GMT'},
  'RetryAttempts': 0}}

# Create endpoint config

In [23]:
config_name = "sm-endpoint-config-sklearn"+ "-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

## Realtime Mode

In [24]:
endpoint_config_resp = sm_client.create_endpoint_config(
                EndpointConfigName=config_name,
                ProductionVariants=[
                    {
                        "VariantName": "primaryvariant",
                        "ModelName": model_name,
                        "InstanceType": 'ml.t2.medium',
                        "InitialInstanceCount": 1
                    },
                ],
    )

In [25]:
endpoint_config_resp

{'EndpointConfigArn': 'arn:aws:sagemaker:ap-southeast-1:405392309789:endpoint-config/sm-endpoint-config-sklearn-2023-06-06-20-32-38',
 'ResponseMetadata': {'RequestId': 'b1bf6823-5e51-4d40-a298-f636ad3969cf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b1bf6823-5e51-4d40-a298-f636ad3969cf',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '132',
   'date': 'Tue, 06 Jun 2023 20:32:40 GMT'},
  'RetryAttempts': 0}}

## Serverless Mode

In [ ]:
endpoint_config_resp = sm_client.create_endpoint_config(
                EndpointConfigName=config_name,
                ProductionVariants=[
                    {
                        "VariantName": "primaryvariant",
                        "ModelName": model_name,
                        "ServerlessConfig": {
                            "MemorySizeInMB": 4096,
                            "MaxConcurrency": 1,
                        },
                    },
                ],
            )

# Deploy endpoint

In [26]:
endpoint_name = "sm-endpoint-sklearn"+ "-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
create_endpoint_resp = sm_client.create_endpoint(
    EndpointName        = endpoint_name,
    EndpointConfigName  = config_name,
)

In [27]:
create_endpoint_resp

{'EndpointArn': 'arn:aws:sagemaker:ap-southeast-1:405392309789:endpoint/sm-endpoint-sklearn-2023-06-06-20-32-45',
 'ResponseMetadata': {'RequestId': '02fc8694-9450-4f02-a8f9-a9800e5e96b8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '02fc8694-9450-4f02-a8f9-a9800e5e96b8',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '112',
   'date': 'Tue, 06 Jun 2023 20:32:45 GMT'},
  'RetryAttempts': 0}}

# Tracking create endpoint process

In [28]:
describe_endpoint_response = sm_client.describe_endpoint(EndpointName=endpoint_name)
while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = sm_client.describe_endpoint(EndpointName=endpoint_name)
    print(f"[LOG]STATUS: {describe_endpoint_response['EndpointStatus']}")
    print(f"[INFO]Working on create endpoint. Hold on tight")
    time.sleep(15)
    
if describe_endpoint_response["EndpointStatus"] == 'Failed':
    print(f"[INFO]Create endpoint {endpoint_name} failed")
else:
    print(f"[INFO]Create endpoint {endpoint_name} successful")

[LOG]STATUS: Creating
[INFO]Working on create endpoint. Hold on tight
[LOG]STATUS: Creating
[INFO]Working on create endpoint. Hold on tight
[LOG]STATUS: Creating
[INFO]Working on create endpoint. Hold on tight
[LOG]STATUS: Creating
[INFO]Working on create endpoint. Hold on tight
[LOG]STATUS: Creating
[INFO]Working on create endpoint. Hold on tight
[LOG]STATUS: Creating
[INFO]Working on create endpoint. Hold on tight
[LOG]STATUS: Creating
[INFO]Working on create endpoint. Hold on tight
[LOG]STATUS: Creating
[INFO]Working on create endpoint. Hold on tight
[LOG]STATUS: Creating
[INFO]Working on create endpoint. Hold on tight
[LOG]STATUS: Creating
[INFO]Working on create endpoint. Hold on tight
[LOG]STATUS: Creating
[INFO]Working on create endpoint. Hold on tight
[LOG]STATUS: Creating
[INFO]Working on create endpoint. Hold on tight
[LOG]STATUS: Creating
[INFO]Working on create endpoint. Hold on tight
[LOG]STATUS: Creating
[INFO]Working on create endpoint. Hold on tight
[LOG]STATUS: Creatin

# Test endpoint invocation

In [29]:
sm_runtime = boto_session.client(service_name='sagemaker-runtime')

In [55]:
data = [5.0, 3.6, 1.4, 0.2]
payload = json.dumps({'Input':data})

In [57]:
response = sm_runtime.invoke_endpoint(
                            EndpointName=endpoint_name, 
                            ContentType='application/json',
                            Body=bytes(payload, 'utf-8')
                            )

In [58]:
assert response['ResponseMetadata']['HTTPStatusCode'] == 200

In [59]:
json.loads(response['Body'].read().decode('utf-8'))

{'Output': 'Setosa'}